# Pretrained Model: EfficientNet-b7

In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from time import time
import math

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

from sklearn.metrics import f1_score

import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize, GaussianBlur, RandomRotation, ColorJitter
from efficientnet_pytorch import EfficientNet
import dataset_hs

In [2]:
import requests 
def send_message_to_slack(text):
    url = "WebHook Url"
    payload = { "text" : text }
    requests.post('', json=payload)

In [3]:
# Set random seed
import random
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

## Augmentation - 우선은 없이 해봄

#### RGB별 평균, 표준편차 구하기

In [4]:
def get_img_mean_std(path):
    """
    RGB 평균 및 표준편차를 수집하는 함수입니다.
    Args:
        path: path_add_label.csv의 위치
    Returns:
        mean, std
    """
    img_info = dict(means=[], stds=[])
    path_label = pd.read_csv(path)
    for img_path in path_label['path']:
        img = np.array(Image.open(os.path.join('./train/train/images',img_path)))
        # (0,1,2)중 0번과 1번에 대해 평균
        # 512, 384 에 대한 평균이 3장 나옴
        img_info['means'].append(img.mean(axis=(0,1)))
        img_info['stds'].append(img.std(axis=(0,1)))
            
    return np.mean(img_info["means"], axis=0) / 255., np.mean(img_info["stds"], axis=0) / 255.

In [5]:
mean, std  = np.array([0.56019358, 0.52410121, 0.501457  ]), np.array([0.23318603, 0.24300033, 0.24567522])

## Dataset정의

In [6]:
class MaskDataset(data.Dataset):
    def __init__(self, img_dir, transform=None):
        """
        img_dir: 학습 이미지 폴더(images)의 root directory(./train/train/images)
        transforms안넣으면 totensor변환만 해서 내보냄
        """
        self.transform = transform
        self.img_dir = img_dir
        self.path = []
        self.label = []
        self.class_num = 18
        self.setup()

    
    def setup(self):
        filenames = os.listdir(self.img_dir)
        for i in filenames:
            if not i.startswith("._"):
                img_name = os.listdir(os.path.join(self.img_dir, i))
                for j in img_name:
                    if not j.startswith('._'):
                        self.path.append(i+'/'+j)
                        gender = 0 if i.split('_')[1] == 'male' else 1
                        age = int(i.split('_')[3])
                        age_range = 0 if age < 30 else 1 if age < 60 else 2
                        if 'incorrect' in j:
                            mask = 1
                        elif 'mask' in j:
                            mask=0
                        elif 'normal' in j:
                            mask=2
                        self.label.append(mask * 6 + gender * 3 + age_range)
                
    def __getitem__(self, index):
        y = self.label[index]
        img_path = self.path[index]
    
        img = Image.open(os.path.join(self.img_dir,img_path))
        if self.transform != None:
            X = self.transform(img)
        else:
            tt = transforms.ToTensor()
            X = tt(img)
        return X, y
        
    def __len__(self):
        return len(self.path)

## train test split

In [7]:
# 정의한 Augmentation 함수와 Dataset 클래스 객체를 생성합니다.

transform = transforms.Compose([
        Resize((int(512 / 2), int(384/ 2))),
        ToTensor(),
        Normalize(mean=mean, std=std)
    ])

dataset = MaskDataset(
    img_dir = '/opt/ml/input/data/train/images',
    transform=transform
)

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

## dataloader만들기

In [8]:
# training dataloader은 데이터를 섞어주어야 합니다. (shuffle=True)
train_loader = data.DataLoader(
    train_dataset,
    batch_size=16,
    num_workers=4, 
#     cuda설정이안돼서 num_workers 설정하면 안돌아감
    shuffle=True
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=16,
    num_workers=4,
    shuffle=False
)
loader={'train':train_loader,
       'val':val_loader}

In [9]:
images, labels = next(iter(train_loader))
print(f'images shape: {images.shape}')
print(f'labels shape: {labels.shape}')

images shape: torch.Size([16, 3, 256, 192])
labels shape: torch.Size([16])


##  모델 불러오기

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
vision_model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=18)
print("네트워크 필요 입력 채널 개수", vision_model._conv_stem.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", vision_model._fc.weight.shape[0])

Loaded pretrained weights for efficientnet-b7
네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 18


In [14]:
# weight및 bias 초기화
torch.nn.init.xavier_uniform_(vision_model._fc.weight)
stdv = 1. / math.sqrt(vision_model._fc.weight.size(1))
vision_model._fc.bias.data.uniform_(-stdv, stdv)

tensor([-0.0121, -0.0100,  0.0074,  0.0148, -0.0026,  0.0135,  0.0026, -0.0176,
        -0.0142, -0.0017,  0.0163, -0.0195,  0.0001, -0.0046,  0.0185,  0.0179,
        -0.0107,  0.0187])

In [16]:
# 학습에 필요한 설정
vision_model.to(device)
LEARNING_RATE = 0.0001 
NUM_EPOCH = 10
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vision_model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')

In [ ]:
best_val_accuracy = 0.
best_val_loss = 9999.

torch.cuda.empty_cache()

for epoch in range(NUM_EPOCH):
    for phase in ["train", "val"]:
        running_loss = 0.
        running_acc = 0.
        n_iter = 0
        epoch_f1 = 0
        if  phase == "val":
            confusion_matrix = np.zeros((18, 18))

        if phase == "train":
            vision_model.train()
        elif phase == "val":
            vision_model.eval()

        for ind, (images, labels) in enumerate(tqdm(loader[phase])):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

            with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산
                logits = vision_model(images)
                _, preds = torch.max(logits, 1)
                loss = loss_fn(logits, labels)

                if phase == "train":
                    loss.backward() 
                    optimizer.step()
            # Metrics 계산 부분 ==============================================================================
            epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            n_iter += 1
            if  phase == "val":
                for t, p in zip(labels.view(-1), preds.view(-1)): # confusion matrix에 값 입력, 언제가 최적일 지 몰라 매 epoch돌아감
                    confusion_matrix[t.long(), p.long()] += 1    
            running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값, images.size(0) = batch size
            running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값
            
            
    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(loader[phase].dataset)
    epoch_acc = running_acc / len(loader[phase].dataset)
    epoch_f1 = epoch_f1/n_iter

    print(f"epoch-{epoch}의 {phase}-데이터 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, 평균 f1:{epoch_f1}")
    if phase == "val" and best_val_accuracy < epoch_acc: 
        best_val_accuracy = epoch_acc
    if phase == "val" and best_val_loss > epoch_loss: 
        best_val_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_val_accuracy:3f}, 최고 낮은 loss : {best_val_loss:3f}")

In [ ]:
# 파일을 저장
from pytz import timezone
import datetime as dt

now = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y-%m-%d-%H-%M"))
torch.save(vision_model,f"save_model/{now}.pth")

In [ ]:
send_message_to_slack(f"학습 완료 되었습니다.{now}")

In [ ]:
plt.figure(figsize=(15,10))

class_names=[str(i) for i in range(18)]
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cmap='YlGnBu')

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=15)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=15)
plt.ylabel('True label')
plt.xlabel('Predicted label')

### predict

In [21]:
model = torch.load(f'save_model/{now}.pth')

In [22]:
class TestDataset(Dataset):
    """
    img_paths: image위치가 들어있는 주소 리스트
    """
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [23]:
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

transform = transforms.Compose([
            Resize((int(512 / 2), int(384/ 2))),
            ToTensor(),
            Normalize(mean=mean, std=std),
        ])
dataset = TestDataset(image_paths, transform)
loader = DataLoader(
    dataset,
    shuffle=False
)

In [24]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = model.to(device)
model.eval()

all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

In [25]:
submission.to_csv(f'submission/submission_5.csv', index=False)